In [48]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize

In [43]:
initial_dataset = pd.read_csv("biodegradable_a.csv")
total_len, _ = initial_dataset.shape

# NOTE - NO INDEPENDENT VALIDATION SET !!!

# Total with means
categorical = ['int16', 'int32', 'int64']
biodegradable = ['object']
numerical = ['float16', 'float32', 'float64']

total_categorical_dataset = initial_dataset.select_dtypes(include=categorical)
total_categorical_dataset = total_categorical_dataset.fillna(total_categorical_dataset.mode())

total_numerical_dataset = initial_dataset.select_dtypes(include=numerical)
total_numerical_dataset = total_numerical_dataset.fillna(total_numerical_dataset.mean())

total_biodegradable = initial_dataset.select_dtypes(include=biodegradable)
total_biodegradable = total_biodegradable.fillna("")

# Scale numerical data
# https://scikit-learn.org/stable/modules/preproce
print(total_numerical_dataset)

scaler = StandardScaler()
#scaler = MinMaxScaler(feature_range=(-1, 1))

total_numerical_dataset = pd.DataFrame(scaler.fit_transform(total_numerical_dataset),
             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)


#total_numerical_dataset = pd.DataFrame(normalize(total_numerical_dataset, norm='l2', axis=1, copy=True, return_norm=False),
#             columns=total_numerical_dataset.columns, index=total_numerical_dataset.index)

print(total_numerical_dataset)
#
total_dataset = pd.concat([total_categorical_dataset, total_numerical_dataset,total_biodegradable], axis=1)

total_len, _ = total_dataset.shape
train_dataset_len = round(total_len * 0.75)

dataset_train = total_dataset[0:train_dataset_len]
dataset_test = total_dataset[train_dataset_len:total_len]

print(total_dataset.shape)

# Removal of None/NaN vals
dropna_dataset = initial_dataset.dropna()

dropna_len, _ = dropna_dataset.shape
model_dropna_len = round(total_len * 0.75)

dropna_train = dropna_dataset[0:model_dropna_len]
dropna_test = dropna_dataset[model_dropna_len:dropna_len]

print(dropna_dataset.shape)

       SpMax_L   J_Dz(e)  nHM       F01  F04  NssssC  nCb          C      nCp  \
0     3.919000  2.690900  0.0  0.000000  0.0     0.0  0.0  31.400000  2.00000   
1     4.170000  2.114400  0.0  0.000000  0.0     0.0  0.0  30.800000  1.00000   
2     3.932000  3.251200  0.0  0.000000  0.0     0.0  0.0  26.700000  2.00000   
3     3.000000  2.709800  0.0  0.000000  0.0     0.0  0.0  20.000000  1.22579   
4     4.236000  3.394400  0.0  0.000000  0.0     0.0  0.0  29.400000  2.00000   
...        ...       ...  ...       ...  ...     ...  ...        ...      ...   
4559  4.175650  3.454649  0.0  0.000000  0.0     0.0  0.0  27.300000  2.00000   
4560  3.853731  2.747142  0.0  0.010373  0.0     0.0  0.0  33.300000  0.00000   
4561  4.294771  3.471226  0.0  0.000000  0.0     0.0  0.0  28.422154  2.00000   
4562  4.560376  3.896390  0.0  0.000000  0.0     0.0  0.0  35.100603  2.00000   
4563  4.045097  2.847185  0.0  0.000000  0.0     0.0  0.0  23.386643  1.00000   

       nO  ...      nCrt  C

In [44]:
print(total_dataset.columns)
print(dropna_test.columns)

Index(['Biodegradable', 'SpMax_L', 'J_Dz(e)', 'nHM', 'F01', 'F04', 'NssssC',
       'nCb', 'C', 'nCp', 'nO', 'F03', 'SdssC', 'HyWi_B', 'LOC', 'SM6_L',
       'F03_CO', 'Me', 'Mi', 'nN_N', 'nArNO2', 'nCRX3', 'SpPosA_B', 'nCIR',
       'B01', 'B03', 'N_073', 'SpMax_A', 'Psi_i_1d', 'B04', 'SdO', 'TI2_L',
       'nCrt', 'C_026', 'F02_CN', 'nHDon', 'SpMax_B', 'Psi_i_A', 'nN', 'SM6_B',
       'nArCOOR', 'nX'],
      dtype='object')
Index(['SpMax_L', 'J_Dz(e)', 'nHM', 'F01', 'F04', 'NssssC', 'nCb', 'C', 'nCp',
       'nO', 'F03', 'SdssC', 'HyWi_B', 'LOC', 'SM6_L', 'F03_CO', 'Me', 'Mi',
       'nN_N', 'nArNO2', 'nCRX3', 'SpPosA_B', 'nCIR', 'B01', 'B03', 'N_073',
       'SpMax_A', 'Psi_i_1d', 'B04', 'SdO', 'TI2_L', 'nCrt', 'C_026', 'F02_CN',
       'nHDon', 'SpMax_B', 'Psi_i_A', 'nN', 'SM6_B', 'nArCOOR', 'nX',
       'Biodegradable'],
      dtype='object')


## Using Model with replaced values when NaN, and discarding the dropped NaN values dataset

In [45]:
X_train = dataset_train.drop(["Biodegradable"], axis=1)
y_train = dataset_train.Biodegradable
print(X_train)
print(y_train)

       SpMax_L   J_Dz(e)       nHM       F01       F04    NssssC       nCb  \
0    -1.320531 -0.666887 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
1    -0.798293 -1.598449 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
2    -1.293482  0.238498 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
3    -3.232628 -0.636346 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
4    -0.660971  0.469894 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
...        ...       ...       ...       ...       ...       ...       ...   
3418  0.495700  0.341900 -0.233216 -0.086545 -0.217059 -0.128692  0.778991   
3419 -0.115755 -0.394899 -0.233216 -0.086545 -0.217059 -0.128692  1.478581   
3420  0.363636  0.106824 -0.233216 -0.086545 -0.217059 -0.128692  0.778991   
3421 -1.150057 -0.041602 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   
3422 -0.672506 -0.520874 -0.233216 -0.086545 -0.217059 -0.128692 -0.620189   

                 C       nCp        nO  ...          nCrt     C

In [46]:
X_test = dataset_test.drop(["Biodegradable"], axis=1)
y_test = dataset_test.Biodegradable

## Testing Ridge with a Linear Model

In [50]:
#ridge = Ridge(alpha=10, max_iter=9999999).fit(X_train, y_train)

#print("The bias is: ",  ridge.intercept_)
#print("The other parameters are: ")
#for i, beta in enumerate(ridge.coef_):
#    print("\t B%02d -> %9.3f"% (i+1, beta))

## Testing lasso with a Linear Model

In [52]:
L = Lasso(alpha=2, max_iter=9999999).fit(X_train, y_train)

print("The bias is: ",  L.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(L.coef_):
    print("\t B%02d -> %9.3f"% (i+1, beta))
preds=L.predict(X_test)
#printRegStatistics(y_test, preds)

ValueError: could not convert string to float: 'RB'